In [3]:
import pandas as pd
import os

# 파일 경로
file_path = 'csv/작물별 Kc.xlsx'

# 엑셀 불러오기
df = pd.read_excel(file_path)

from datetime import datetime

# 1. 사용자 입력
start_date = input("재배 시작 날짜 (예: 2024-04-15): ")
today = input("오늘 날짜 입력 (예: 2024-05-20): ")
crop = input("작물명 입력 (예: 당근): ")

# 2. 날짜 차이 계산
start_dt = datetime.strptime(start_date, "%Y-%m-%d")
today_dt = datetime.strptime(today, "%Y-%m-%d")
days_passed = abs((today_dt - start_dt).days)
print(f"경과일수: {days_passed}일")

# 3. 작물 정보 불러오기
crop_row = df[df["작물 종류"] == crop].iloc[0]
total_days = crop_row["생육일수"]

# 4. 단계 구간 계산
ini = int(total_days * 0.2)
mid = int(total_days * 0.6)
end = total_days - ini - mid

# 5. 현재 단계 판단
if days_passed <= ini:
    stage = "ini"
    kc = crop_row["Kc_ini"]
elif days_passed <= ini + mid:
    stage = "mid"
    kc = crop_row["Kc_mid"]
else:
    stage = "end"
    kc = crop_row["Kc_end"]

import requests
import xml.etree.ElementTree as ET
import time

# 1) 주소 → 위경도 변환
def get_coords_from_vworld(address, api_key):
    url = "https://api.vworld.kr/req/address"
    params = {
        'service': 'address',
        'version': '2.0',
        'request': 'GetCoord',
        'format': 'json',
        'key': api_key,
        'type': 'ROAD',   # ROAD / PARCEL / BOTH
        'address': address,
        'crs': 'EPSG:4326'
    }
    res = requests.get(url, params=params, timeout=10)
    res.raise_for_status()
    resp = res.json().get('response', {})
    if resp.get('status') != 'OK':
        raise RuntimeError(f"VWorld 주소 변환 오류: {resp.get('error')}")
    result = resp['result']
    items = result.get('items') or result.get('item')
    if items:
        feat = items[0] if isinstance(items, list) else items
        point = feat['point']
    else:
        point = result.get('point')
    return float(point['x']), float(point['y'])


# 2) 위경도로 배수등급 조회 & λ 매핑
def fetch_soil_drainage_with_lambda(lon, lat, data_api_key):
    # 2-1) 원격 호출
    url = 'https://api.vworld.kr/req/data'
    params = {
        'key': data_api_key,
        'service': 'data',
        'request': 'GetFeature',
        'data': 'LT_C_ASITSOILDRA',
        'format': 'json',
        'geometry': 'true',
        'page': '1',
        'size': '1000',
        'crs': 'EPSG:4326',
        'domain': 'localhost',
        'geomFilter': f'POINT({lon} {lat})'
    }
    res = requests.get(url, params=params, timeout=10)
    res.raise_for_status()
    feats = res.json()['response']['result']['featureCollection']['features']
    
    # 2-2) DataFrame 변환
    soil_df = pd.DataFrame([f['properties'] for f in feats])
    
    # 2-3) 등급 → λ 매핑 테이블
    drain_to_lambda = {
        '매우양호': 0.50,
        '양호':     0.35,
        '약간양호': 0.25,
        '약간불량': 0.15,
        '불량':     0.10,
        '매우불량': 0.05,
    }
    
    # 2-4) 'label' 컬럼 기준으로 λ 추가
    soil_df['lambda'] = soil_df['label'].map(drain_to_lambda)
    return soil_df[['label', 'lambda']]


if __name__ == '__main__':
    # 사용자 입력
    address      = input("📍 조회할 주소: ")
    vworld_addr  = 'B4C5E64B-E7B8-3103-9156-289899AE4279'  # 주소 API 키
    vworld_data  = '5EE19860-B1C7-3082-B027-99245C4FC9BE'  # 데이터 API 키

    # 주소 → 위경도
    lon, lat = get_coords_from_vworld(address, vworld_addr)
    soil_df = fetch_soil_drainage_with_lambda(lon, lat, vworld_data)
    print(f"▶ 변환 좌표: {address} -> 경도={lon:.6f}, 위도={lat:.6f}")

    # 배수등급 조회 + λ 매핑
    result = fetch_soil_drainage_with_lambda(lon, lat, vworld_data)
    print("✅ 배수등급 ↔ 손실계수 λ")
    print(result.to_string(index=False))
    
    
    
    
    
#------
service_key = 'OaE7WFXPyKXCPtSvtE9HuQdSwbzhl/C9FhjkxVzyOfKLRZxqAMChtLhArevfCux2XuluPYLtgDuMUEPXvGaoNQ=='
url = 'http://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList'

params ={'serviceKey' : service_key, 
         'pageNo' : '1', 
         'numOfRows' : '100', 
         'dataType' : 'XML', 
         'dataCd' : 'ASOS', 
         'dateCd' : 'DAY', 
         'startDt' : '20240301', 
         'endDt' : '20240901', 
         'stnIds' : '108' }



all_data = []
page = 1

while True:
    print(f'Fetching page {page}...')
    params = params.copy()
    params['pageNo'] = str(page)
    
    response = requests.get(url, params=params)
    root = ET.fromstring(response.content)

    print(response.content)

    items = root.findall('.//item')
    if not items:
        print('✅ All data fetched.')
        break
    
    for item in items:
        all_data.append({
            'date': item.findtext('tm'),
            'avg_temp': item.findtext('avgTa'),
            'min_temp': item.findtext('minTa'),
            'max_temp': item.findtext('maxTa'),
            'rainfall': item.findtext('sumRn'),
            'humidity': item.findtext('avgRhm'),
            'sunshine': item.findtext('sumGsr'),
            'wind': item.findtext('avgWs'),
            'air_pressure': item.findtext('avgPa')
        })
    
    page += 1
    time.sleep(0.3)

# DataFrame 생성
df = pd.DataFrame(all_data)

df['date'] = pd.to_datetime(df['date'])
df = df.sort_values('date').reset_index(drop=True)

for col in ['avg_temp', 'min_temp', 'max_temp', 'rainfall', 'humidity', 'sunshine', 'wind', 'air_pressure']:
    df[col] = pd.to_numeric(df[col], errors='coerce')

df['rainfall'].fillna(0, inplace = True)

# 인덱스를 파일에 포함시키지 않을 때
os.makedirs('results', exist_ok=True)
output_path = os.path.join('results', 'weather.csv')
df.to_csv(output_path, index=False, encoding='utf-8-sig')
print(f'\n✔ Saved weather results to {output_path}')



# --- 위에서 정의한 함수 --- 
def calculate_irrigation(df, 
                         lam,
                         kc,# 배수 손실계수 λ
                         init_res=0   # t=0일 전 잔존수분
                        ):
    df = df.copy()
    df['date'] = pd.to_datetime(df['date'])
    for col in ['avg_temp','min_temp','max_temp','rainfall','sunshine']:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    df['rainfall'].fillna(0, inplace=True)

    def _eto(r):
        if pd.notna(r['avg_temp']) and pd.notna(r['min_temp']) \
           and pd.notna(r['max_temp']) and pd.notna(r['sunshine']):
            return 0.0023 * (r['avg_temp']+17.8) \
                   * ((r['max_temp']-r['min_temp'])**0.5) * r['sunshine']
        return 0
    df['ETo'] = df.apply(_eto, axis=1)
    df['ETc'] = df['ETo'] * kc
    df['Pe']  = df['rainfall'] * 0.8

    df['irrigation'] = 0.0
    df['residual']   = 0.0

    res_prev = init_res

    for idx, row in df.iterrows():
        P  = row['Pe']
        ET = row['ETc']

        I = max(0, ET - (P + res_prev) + lam*(P + res_prev))
        res_t = max(0, res_prev + P + I - ET - (1-lam)*(P + I))

        df.at[idx, 'irrigation'] = I
        df.at[idx, 'residual']   = res_t

        res_prev = res_t

    return df[['date','avg_temp','min_temp','max_temp',
               'rainfall','ETo','ETc','Pe','irrigation','residual']]


# --- 스크립트 메인 흐름 ---
if __name__ == '__main__':
    # 1) 입력: weather.csv 읽기
    df_weather = pd.read_csv('results/weather.csv')   # 실제 파일명에 맞게 수정

    # 2) 관개량/잔존수분 계산
    df_result = calculate_irrigation(
        df_weather,
        kc=kc,    # 작물별 Kc
        lam = soil_df['lambda'].iloc[0],   # 배수 손실계수 λ
        init_res=0  # 초기 잔존수분
    )


    # 3) 결과를 CSV로 저장
    output_path = os.path.join('results', 'irrigation_results.csv')
    df_result.to_csv(output_path, index=False, encoding='utf-8-sig')
    print(f'\n✔ Saved results to {output_path}')

df_result

ImportError: Pandas requires version '3.1.0' or newer of 'openpyxl' (version '3.0.10' currently installed).